In [13]:
import sys
from matplotlib import pyplot as plt
sys.path.append("..")
import datetime
import pandas as pd
import time
import os
from Training.Main import *
from scipy.stats import pearsonr

def calculate_pvalues(df):
    # Code from https://stackoverflow.com/questions/25571882/pandas-columns-correlation-with-statistical-significance
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            # pvalues[r][c] = (pearsonr(df[r], df[c])[1], 4)
            pvalues[r][c] = pearsonr(df[r], df[c])[1]
    return pvalues


In [14]:
FEATURES = [
    'Volume',
    'tweet_vol',
    'sent_neg',
    'sent_neu',
    'sent_pos',
    "sent_compound",
    'count',
    'count_neg',
    'count_neu',
    'count_pos',
    'Close',
    "High",
    "Low",
]
TIME_INTERVALS = [
    "10Min", "30Min", "1H", "2H",
    "3H", "6H",
    "12H",
    "24H"
]

In [15]:
corr_matrix  = pd.DataFrame()
corr_matrix_next  = pd.DataFrame()
corr_matrix_td  = pd.DataFrame()
corr_matrix_next_td  = pd.DataFrame()

for time_interval in TIME_INTERVALS:
    df = load_data(time_interval)

    # bert_vector = []
    # for i in range(0, 768):
    #     bert_vector.append("bert" + str(i))
    # df_bert = df[bert_vector]

    df = df[FEATURES]
    # df = pd.concat([df[FEATURES], df_bert], axis=1)


    df = time_differencing(df)

    df["Close_td_next"] = df["Close"].shift(-1)
    df["Close_next"] = df["Close_v"].shift(-1)
    df = df.iloc[:-1]


    corr = df.corr(method='pearson')
    p = calculate_pvalues(df)
    print("time_interval:",time_interval)
    corr_matrix_td[time_interval] = corr["Close"].round(4)
    corr_matrix_next_td[time_interval] = corr["Close_td_next"].round(4)
    corr_matrix_next[time_interval] = corr["Close_next"].round(4)
    corr_matrix[time_interval] = corr["Close_v"].round(4)

    corr_matrix_td[time_interval+"p"] = p["Close"]
    corr_matrix_next_td[time_interval+"p"] = p["Close_td_next"]
    corr_matrix_next[time_interval+"p"] = p["Close_next"]
    corr_matrix[time_interval+"p"] = p["Close_v"]

READING TWEETS
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/TWEETS(10Min).csv
READING COMPOUND
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/COMPOUND(10Min).csv
READING VOLUME
 - Cached File Found
READING BITCOIN
 - Cached File Found
READING SENTIMENT VOLUME
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/SENT_VOLUME(10Min).csv
DATA READY
time_interval: 10Min
READING TWEETS
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/TWEETS(30Min).csv
READING COMPOUND
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/COMPOUND(30Min).csv
READING VOLUME
 - Cached File Found
READING BITCOIN
 - Cached File Found
READING SENTIMENT VOLUME
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/SENT_VOLUME(30Min).csv
DATA READY
time_interval: 30Min
READING TWEETS
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/TWEETS(1H).csv
READING COMPOUND
 - Cached File Found
 - LOADED ../Data/2018-Weighted/cache/COMPOUND(1H).csv
READING VOLUME
 - 

In [16]:
print("corr_matrix")
print(corr_matrix)
print("corr_matrix_next")
print(corr_matrix_next)
print("corr_matrix_td")
print(corr_matrix_td)
print("corr_matrix_next_td")
print(corr_matrix_next_td)

corr_matrix
                10Min        10Minp   30Min        30Minp      1H  \
Volume         0.1985  3.22015e-305  0.2466  7.92242e-160  0.2820   
tweet_vol      0.0304   1.46154e-08  0.0491   1.27497e-07  0.0597   
sent_neg      -0.0182   0.000699904 -0.0286    0.00205907 -0.0406   
sent_neu       0.0469   2.28848e-18  0.0583    3.5268e-10  0.0726   
sent_pos      -0.0432   8.57619e-16 -0.0483   2.03448e-07 -0.0542   
sent_compound -0.0339   2.75126e-10 -0.0343   0.000220307 -0.0349   
count          0.3446             0  0.4108             0  0.4381   
count_neg      0.2162             0  0.2586  4.03715e-176  0.2792   
count_neu      0.3426             0  0.4400             0  0.4850   
count_pos      0.3147             0  0.3670             0  0.3902   
Close          0.0075      0.162961  0.0125      0.179583  0.0182   
High           0.9999             0  0.9996             0  0.9993   
Low            0.9999             0  0.9996             0  0.9992   
Close_v        1.0000 

In [17]:
corr_matrix.to_csv('sent_corr.csv')
corr_matrix_next.to_csv('sent_corr_next.csv')
corr_matrix_td.to_csv('sent_corr_td.csv')
corr_matrix_next_td.to_csv('sent_corr_next_td.csv')

In [18]:
pearsonr(df["Volume"], df["Close_v"])

(0.49723930181108467, 2.1524647366132273e-16)